In [7]:
pip install xgboost

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.9/1.9 MB 13.4 MB/s eta 0:00:00a 0:00:01

[notice] A new release of pip is available: 24.0 -> 24.3.1
[notice] To update, run: pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


In [26]:
import numpy as np
import os
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report, accuracy_score

from sklearn.preprocessing import LabelEncoder, StandardScaler
from sklearn.ensemble import RandomForestRegressor, RandomForestClassifier
from sklearn.metrics import mean_squared_error, r2_score
from sklearn.pipeline import Pipeline

import joblib
from playstore.config import CLEAN_DATA_DIR

In [2]:
# Load data
try:
    df = pd.read_csv("cleaned_googleplaystore.csv")  # Update to your actual file path if different
    print("Loaded")
except Exception as e:
    print("Error loading data:", e)
    df = pd.DataFrame({
        'Category': ['Game', 'Education', 'Business'],
        'Rating': [4.5, 4.0, 3.8],
        'Type': ['Free', 'Free', 'Paid'],
        'Installs': [1000000, 500000, 100000],
    })


Loaded


In [30]:
# Rozdelenie hodnotení do kategórií
df['Rating_category'] = pd.cut(
    df['Rating'],
    bins=[0, 3.5, 4.5, 5],
    labels=['Low', 'Medium', 'High']
)

# Pripraviť vlastnosti a cieľovú premennú
X = df[features]  # Rovnaké vlastnosti ako predtým
y = df['Rating_category']

# Encode cieľovú premennú
le = LabelEncoder()
y_encoded = le.fit_transform(y)

# Train-test split
X_train, X_test, y_train, y_test = train_test_split(X, y_encoded, test_size=0.2, random_state=42)

# Trénovanie Random Forest klasifikátora
clf = RandomForestClassifier(
    n_estimators=100,
    max_depth=10,
    random_state=42
)
clf.fit(X_train, y_train)

# Predikcia
train_pred = clf.predict(X_train)
test_pred = clf.predict(X_test)

# Vyhodnotenie modelu
print("Train Accuracy:", accuracy_score(y_train, train_pred))
print("Test Accuracy:", accuracy_score(y_test, test_pred))
print("\nClassification Report:")
print(classification_report(y_test, test_pred, target_names=le.classes_))
feature_importances = clf.feature_importances_

# Vytvorenie tabuľky dôležitosti vlastností
importance_df = pd.DataFrame({
    'Feature': features,
    'Importance': feature_importances
}).sort_values(by='Importance', ascending=False)

# Výpis tabuľky
print(importance_df)


Train Accuracy: 0.7816544738323016
Test Accuracy: 0.703037120359955

Classification Report:
              precision    recall  f1-score   support

        High       0.53      0.19      0.28       374
         Low       0.36      0.02      0.04       178
      Medium       0.72      0.96      0.82      1226

    accuracy                           0.70      1778
   macro avg       0.54      0.39      0.38      1778
weighted avg       0.64      0.70      0.63      1778

            Feature  Importance
0       log_Reviews    0.322115
1          log_Size    0.185995
3  Category_encoded    0.142067
8       Updated_Day    0.123462
7     Updated_Month    0.088654
6      Updated_Year    0.058020
5       interaction    0.039681
2             Price    0.032238
4      Type_encoded    0.007768


In [31]:
model_path = os.path.join(os.getcwd(), "rf_model_cl.pkl")
joblib.dump(model, model_path)
print(f"Model saved to {model_path}")

Model saved to /Users/steve/PycharmProjects/uu_school/playstore/notebooks/rf_model_cl.pkl
